In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot")
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot")

/Users/b.goecmen/miniconda3/envs/ip-copyright-llms/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
code = "def hello_world():\n    print('Hello, World!')\n"
prompt = "def hello_world():"
code2 = """\"\"\"Provides utilities to preprocess images for the Inception networks.\"\"\"

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from tensorflow.python.ops import control_flow_ops

def distort_color(image, color_ordering=0, fast_mode=True, scope=None):
  \"\"\"Distort the color of a Tensor image.

Each color distortion is non-commutative and thus ordering of the color ops
matters. Ideally we would randomly permute the ordering of the color ops.
Rather then adding that level of complication, we select a distinct ordering
of color ops for each preprocessing thread.

Args:
image: 3-D Tensor containing single image in [0, 1].
color_ordering: Python int, a type of distortion (valid values: 0-3).
fast_mode: Avoids slower ops (random_hue and random_contrast)
scope: Optional scope for name_scope.
Returns:
3-D Tensor color-distorted image on range [0, 1]
Raises:
ValueError: if color_ordering not in [0, 3]
\"\"\"
with tf.name_scope(scope, 'distort_color', [image]):
  if color_ordering == 0:
    image = tf.image.random_brightness(image, max_delta=32. / 255.)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
  elif color_ordering == 1:
    image = tf.image.random_brightness(image, max_delta=32. / 255.)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=0.2)
  elif color_ordering == 2:
    image = tf.image.random_brightness(image, max_delta=32. / 255.)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=0.2)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
  elif color_ordering == 3:
    image = tf.image.random_brightness(image, max_delta=32. / 255.)
    image = tf.image.random_hue(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
  else:
    raise ValueError('color_ordering must be in [0, 3]')

  if fast_mode:
    image = tf.clip_by_value(image, 0.0, 1.0)
  return image"""

prompt2 = """\"\"\"Provides utilities to preprocess images for the Inception networks.\"\"\"

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from tensorflow.python.ops import control_flow_ops

def distort_color(image, color_ordering=0, fast_mode=True, scope=None):
 \"\"\"Distort the color of a Tensor image.

Each color distortion is non-commutative and thus ordering of the color ops
matters. Ideally we would randomly permute the ordering of the color ops.
Rather then adding that level of complication, we select a distinct ordering
of color ops for each preprocessing thread.

Args:
image: 3-D Tensor containing single image in [0, 1].
color_ordering: Python int, a type of distortion (valid values: 0-3).
fast_mode: Avoids slower ops (random_hue and random_contrast)
scope: Optional scope for name_scope.
Returns:
3-D Tensor color-distorted image on range [0, 1]
Raises:
ValueError: if color_ordering not in [0, 3]
\"\"\""""

prompt3= '\n"""Various utility functions"""\n\nfrom __future__ import division\n\nimport re\nfrom decimal import Decimal\nfrom math import ceil, cos, floor, log10, pi, sin\n\nfrom pygal._compat import _ellipsis, to_unicode, u\n\n\ndef majorize(values):\n    """Filter sequence to return only major considered numbers"""'
code3 = '\n"""Various utility functions"""\n\nfrom __future__ import division\n\nimport re\nfrom decimal import Decimal\nfrom math import ceil, cos, floor, log10, pi, sin\n\nfrom pygal._compat import _ellipsis, to_unicode, u\n\n\ndef majorize(values):\n    """Filter sequence to return only major considered numbers"""\n    return [v for v in values if v % 1 == 0 or v.is_integer()]'


In [7]:
def to_tokens_and_probs(model, tokenizer, input_texts, as_log_probs=False):
    input_ids = tokenizer(input_texts, padding=True, return_tensors="pt").input_ids
    outputs = model(input_ids)
    if as_log_probs:
        probs = torch.log_softmax(outputs.logits, dim=-1).detach()
    else:
        probs = torch.softmax(outputs.logits, dim=-1).detach()

    # collect the probability of the generated token -- probability at index 0 corresponds to the token at index 1
    probs = probs[:, :-1, :]
    input_ids = input_ids[:, 1:]
    gen_probs = torch.gather(probs, 2, input_ids[:, :, None]).squeeze(-1)

    batch = []
    for input_sentence, input_probs in zip(input_ids, gen_probs):
        text_sequence = []
        for token, p in zip(input_sentence, input_probs):
            if token not in tokenizer.all_special_ids:
                text_sequence.append((tokenizer.decode(token), p.item()))
        batch.append(text_sequence)
    return batch

In [3]:
from transformers import GenerationConfig
from perplexity import Perplexity

perplexity = Perplexity(model=model, tokenizer=tokenizer, device='cpu', use_chat_template=True,
                        generation_config=GenerationConfig(
                            # stop_strings],  # Stop generation upon encountering two consecutive newlines
                            max_new_tokens=512,  # Maximum number of tokens to generate
                            pad_token_id=tokenizer.eos_token_id,
                            tokenizer=tokenizer,
                            use_cache=False
                        ), batch_size=3
                        )

In [5]:
test = perplexity.compute([prompt, prompt2, prompt3], [code, code2, code3], add_start_token=True)

100%|██████████| 1/1 [07:54<00:00, 474.90s/it]


In [ ]:
result[0]["generated_text"].split("### Response")[-1],

In [3]:
from transformers import GenerationConfig

from generation import CodeGenerator
generator = CodeGenerator(model=model, tokenizer=tokenizer, generation_config=GenerationConfig(
    # stop_strings],  # Stop generation upon encountering two consecutive newlines
    max_new_tokens=5,  # Maximum number of tokens to generate
    pad_token_id=tokenizer.eos_token_id,
    tokenizer=tokenizer,
    use_cache=False
), batch_size=2)


Device set to use mps:0


In [5]:
generator.generate_text_with_chat_template(prompts=[prompt, prompt2], language="python")

Generating text: 2it [00:07,  3.97s/it]                       


[GeneratedCode(prompt='def hello_world():', generated_code=''),
 GeneratedCode(prompt='"""Provides utilities to preprocess images for the Inception networks."""\n\nfrom __future__ import absolute_import\nfrom __future__ import division\nfrom __future__ import print_function\n\nimport tensorflow as tf\n\nfrom tensorflow.python.ops import control_flow_ops\n\ndef distort_color(image, color_ordering=0, fast_mode=True, scope=None):\n """Distort the color of a Tensor image.\n\nEach color distortion is non-commutative and thus ordering of the color ops\nmatters. Ideally we would randomly permute the ordering of the color ops.\nRather then adding that level of complication, we select a distinct ordering\nof color ops for each preprocessing thread.\n\nArgs:\nimage: 3-D Tensor containing single image in [0, 1].\ncolor_ordering: Python int, a type of distortion (valid values: 0-3).\nfast_mode: Avoids slower ops (random_hue and random_contrast)\nscope: Optional scope for name_scope.\nReturns:\n3-D